### Script
- **Input:** scripts to run traditional recommendation using content-based, and collaborative filtering approaches.
- **Output:** Sample recommendations.

### Import libraries

In [1]:
!pip install gensim
import gensim
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, Word2Vec
from gensim.test.utils import get_tmpfile

### Load data

In [2]:
root = './RASA_realtime_recommendation/data/'
model_root='./RASA_realtime_recommendation/offline_models/'
processed = './Data/processing/Processed_Airbnb/'
raw = './Data/raw/'

### Recommend by user review comments
- https://towardsdatascience.com/detecting-document-similarity-with-doc2vec-f8289a9a7db7

In [12]:
reviews = pd.read_csv(processed+'ratings_filter.csv', sep=',')
vector_file_name = root+"embeddings/review_embeddings"
def recommendListingByUserComments(reviews,vector_file_name,topK):
    result=[]
    
    comment_corpus_model = Doc2Vec.load(vector_file_name) 
    new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher safety")
    test_doc_vector = comment_corpus_model.infer_vector(new_doc)
    sims = comment_corpus_model.dv.most_similar(positive = [test_doc_vector])
    print(sims)
    for s in sims[:topK+1]:
        try:
            result.append(reviews['listing_id'].iloc[s[0]])
        except:
            print()
    return result

print('Top-5 recommended listing ids:')
print(recommendListingByUserComments(reviews,vector_file_name,5))

Top-5 recommended listing ids:
[(179370, 0.5591686367988586), (180654, 0.5491060018539429), (196714, 0.5471900105476379), (238022, 0.5451457500457764), (135392, 0.5433078408241272), (242540, 0.5373138785362244), (242023, 0.5343819260597229), (187580, 0.5341843962669373), (205352, 0.5338578820228577), (191352, 0.525367259979248)]
['17875133', '18059130', '20735962', '31871461', '11165921', '33448591']


### Recommend by listing vector properties

In [7]:
listings = pd.read_csv(raw+'listings.csv.gz', sep=',')
vector_file_name = root+"embeddings/list_embeddings"
def recommendListingByProperties(listings,vector_file_name,topK):
    result=[]
    
    comment_corpus_model = Doc2Vec.load(vector_file_name) 
    new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher safety")
    test_doc_vector = comment_corpus_model.infer_vector(new_doc)
    sims = comment_corpus_model.docvecs.most_similar(positive = [test_doc_vector])
    print(sims)
    for s in sims[:topK]:
        result.append(reviews['listing_id'].iloc[s[0]])
    return result

print('Top-5 recommended listing ids:')
print(recommendListingByUserComments(reviews,vector_file_name,5))

Top-5 recommended listing ids:
[(3128, 0.3928413987159729), (3487, 0.38972726464271545), (4761, 0.3859352469444275), (57, 0.3849267363548279), (4763, 0.3827367126941681), (4756, 0.377719908952713), (4760, 0.37745940685272217), (4759, 0.3764130473136902), (4752, 0.374215692281723), (44, 0.3740732669830322)]
['80635', '82482', '118097', '2818', '118097', '118097']


### Recommend by listings - ContentBased filtering

In [21]:
model = Word2Vec.load(model_root+'ContentBasedFilter')
watch = listings[["id","listing_url","name"]]

# create movie id and tittle dictionary
watch_dict = watch.groupby('id')['id'].apply(list).to_dict()
# print(watch_dict)
def similar_watch(v, n = 5):

    # extract most similar movies for the input vector
    ms = model.wv.similar_by_vector(v, topn= n+1)[1:]
    # extract name and similarity score of the similar movies
    
    return ms  


topK=5
ls = similar_watch(2818,topK)
print('Top-'+str(topK)+' recommendations:')
for j in ls:
    print(j[0])
   
        

KeyError: "Key '2818.0' not present"

### Recommend by listings - Collaborative filtering

In [5]:
# https://sigdelta.com/blog/how-to-install-pyspark-locally/

In [8]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALSModel

def recommendByALSCollabFilter(topK,user_id):

    spark = SparkSession.builder.getOrCreate()
    sc = spark.sparkContext
    alsmodel = ALSModel.load("/Users/sudhavijayakumar/Documents/299/299A-SMARTRec/RASA/offline_models/als_model")
    # convert this into a dataframe so that it can be passed into the recommendForUserSubset
    functiondf = sc.parallelize(user_id).toDF(['reviewer_id'])

    recommendations = alsmodel.recommendForUserSubset(functiondf , topK)
    recommendations.collect()

    recommended_listings = [recommendations.collect()[0]['recommendations'][x]['listing_id'] for x in range(0,topK)]
    return recommended_listings


user_id = [[164729]]
sims = recommendByALSCollabFilter(5,user_id)

# Get first 5 matches
listingss=[]
for s in sims:
	listingss.append('https://www.airbnb.com/rooms/'+str(s))
listingss

Py4JJavaError: An error occurred while calling o23.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/sudhavijayakumar/Documents/299/299A-SMARTRec/RASA/offline_models/als_model/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.R